# Sessionクラス解説

Session クラスはAgentの各イベント(on_tick, on_clock, on_update)時に引数として送られてきます。
このSessionのインスタンスを利用してAgent内の取引ロジックを開発してください。
使えるプロパティ／メソッドについて説明します。

このNoteの中では説明のためにSessionオブジェクトのインスタンス化や内部関数も使いますが、
ユーザがはSessionオブジェクトを自分で生成することはありません。

資金管理などのメソッドは今後の開発予定になっています。

## プロパティ


### 市況を示すプロパティ

<table>
<tr><td >内容</td><td>プロパティ</td></tr>
<tr><td>現在時刻</td><td>Session.current_timestamp</td></tr>
<tr><td>最終売Take価格 = 最終買Make価格</td><td>Session.best_buy_price</td></tr>
<tr><td>最終買Take価格 = 最終売Make価格</td><td>Session.best_sell_price</td></tr>
</table>

### 自分のオーダー／ポジションを表すプロパティ


#### 注残

<table>
<table>
<tr><td></td><td>内容</td><td>関数</td></tr>
<tr><td colspan='3'>Long</td></tr>
<tr><td></td><td>買注文個数</td><td>Session.long_order_len</td></tr>
<tr><td></td><td>買注文一覧</td><td>Session.long_orders</td></tr>
<tr><td colspan='3'>Short</td></tr>
<tr><td></td><td>売注文個数</td><td>Session.short_order_len</td></tr>
<tr><td></td><td>売注文一覧</td><td>Session.short_orders</td></tr>
</table>

#### ポジション

<table>
<tr><td></td><td>内容</td><td>関数</td></tr>
<tr><td colspan='3'>Long</td></tr>
<tr><td></td><td>平均単価</td><td>Session.long_position_price</td></tr>
<tr><td></td><td>サイズ</td><td>Session.long_position_size</td></tr>
<tr><td colspan='3'>Short</td></tr>
<tr><td></td><td>平均単価</td><td>Session.short_position_price</td></tr>
<tr><td></td><td>サイズ</td><td>Session.short_position_size</td></tr>
</tbody></table>
</table>



## 関数

#### オーダー発行(`place_order`)

```
Session.place_order(
    order_side,     # 'Buy'　または 'Sell'を指定
    price,          # 任意の価格が設定できるが相手の板にぶつけるtakeになる価格でもmakeの処理・手数料で処理している。
                    # ギリ指しするために　Session.best_buy_price, Session.best_sell_priceが使える。
    size,           # オーダーサイズ BTCBUSDの場合BTC建で指定。
    duration_sec,   # オーダーの有効期限（秒）。オーダー後、この秒数をこえるとExpireする。
    message         # あとでログで識別できるように任意の文字列が設定できる。
)
```

#### 現在時刻のohlcv取得(`ohlcv`)

```
Session.ohlcv(
    time_window,    #   OHLCVの足の幅（秒）
    num_of_bars,    #   取得する本数
)
```



## 以下テストを兼ねたSessionオブジェクトの内部動作確認

In [28]:
# 通常ユーザがSessionオブジェクトを生成する必要はありません。説明のために内部関数をつかって生成します。
from rbot import Session, _DummySession

session = Session(_DummySession('BN', "BTCBUSD", False))

In [29]:
# 初期状態では中身は空っぽ
session

In [30]:
# 内部処理、バックテスト用にログを読み取らせると市況がUpdateされる。
from rbot import NOW
import rbot

# 最後に買われた価格が 売り板の最良値（best_sell_price)になる。
session.trade(NOW(), "Buy", 100, 1, "ORDERID-1")    # BackTester内部処理
session

In [31]:
# 最後に売られた価格が買い板の最良値(best_buy_price)になる、
session.trade(NOW(), "Sell", 99.9, 1, "ORDERID-2") # BackTester内部処理
session

In [32]:
# Agent用メソッド：　オーダーを発行すると,処理中オーダーにキューされる
session.place_order("Buy", 
                    100,    #価格
                    50,     # サイズ
                    1,     # 有効期間
                    'test order')
session

In [33]:
# 有効期間が1秒なので、1秒まってバックテスターが次のTradeを処理するとオーダーはExpireしてしまう（オーダーがセッションから消える）。
import time
time.sleep(1)
session.trade(NOW(), "Buy", 100, 1, "ORDERID-1")    # BackTester内部処理
session

In [34]:
# Agent用メソッド：　今度は有効期間のながいオーダーを出す。
session.place_order("Buy", 
                    100,    #価格
                    50,      # サイズ
                    60*60,     # 有効期間(1時間)
                    'test order')
session

In [35]:
# 買オーダー(100)よりも高い売り(101.0)がおこなわれてもオーダーには影響がない。
session.trade(NOW(), "Sell", 101.0, 1, "ORDERID-2")  # Back Tester内部処理
session

In [37]:
# 買オーダー（１００）と同じ価格の売りがおこなわれているときも影響がない
# 本来は時間が経って板の最後までまわってきたら約定するが、辛めに判定
session.trade(NOW(), "Sell", 100.0, 1, "ORDERID-2")  # Back Tester内部処理
session

In [38]:
# 買オーダー(100)よりも低い価格の売りがでてきたら、部分約定する（今回の例では、サイズ１が約定して49が残っている）
# ただし約定ログは分割せずremain_sizeをへらして全部約定のときにログを出す（あとで、まとめて約定したことにする）
session.trade(NOW(), "Sell", 99.9, 1, "ORDERID-2")  # Back Tester内部処理
session

In [41]:
# のこり49の売りが発生すると全部約定し、オーダーがlongポジションへ移動する。
session.trade(NOW(), "Sell", 99.9, 49, "ORDERID-2")  # Back Tester内部処理
session

In [42]:
# Agent用メソッド：　ここでドテンオーダーを出すと売りのオーダーにキューされる。
session.place_order("Sell", 
                    100,    #価格
                    100,      # サイズ
                    60*60,     # 有効期間(1時間)
                    'test order')
session

In [43]:
# 全部を約定するようなログがきたら、ロングポジション５０が精算されると同時に分割されてあらたに５０のショートポジションが積まれる。
session.trade(NOW(), "Buy", 100.1, 100, "ORDERID-2")  # Back Tester内部処理
session

# まとめ

以上のような動きを約定ログ全部なめてバックテストを行なっています。
（こんな面倒な方法やらなきゃよかった。。。）